In [1]:
import pandas as pd
import numpy as np
import pickle
import json
from flask import  Flask, request, jsonify
from flask_cors import CORS

In [2]:
# cargamos los datos Capital Federal --> features extras de cercanias por barrios
dfbarrios = pd.read_csv('df41e_media_geograficas_por_barrios.csv', index_col='index',encoding='utf-8')

# cargamos el modelo
modelo = pickle.load(open("modelo.pkl", "rb"))

# cargamos el preprocesador
preprocesador = pickle.load(open("preprocesador.pkl", "rb"))

In [3]:
# creamos la app y habilitamos los CORS (para poder realizar las request desde el HTML propio sin conflicto)
app = Flask('Estimator')
CORS(app)

#### Primero correr las dos siguiente celdas (servicio y run de la app)
### Usar el Estimador.html para acceder al Estimador y testear

In [4]:
# creamos el servicio
@app.route('/api/estimador',methods=['POST'])
def predict2():
    # obtengo los datos del request post
    data = request.get_json(force=True) 
    
    # lo convierto en un df
    df_req = pd.DataFrame(data, index=range(1))
    
    # tomo las distancias que corresponden al barrio
    barrio = dfbarrios.loc[[df_req.loc[0,'place_name']],:]
    barrio['place_name'] = barrio.index
    
    # juntamos la data de ambos DF
    df = df_req.merge(barrio, left_on='place_name', right_on='place_name', suffixes=('_left', '_right'))
    df['surface_total_in_m2_t'] = 1/np.sqrt(int(df['surface_total_in_m2']))
    
    # corremos la prediccion
    result = modelo.predict(preprocesador.transform(df[0:1]))
    
    return jsonify({'result': result[0].round(1)})

In [ ]:
app.run(host='0.0.0.0')

 * Serving Flask app "Estimator" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [07/May/2019 10:14:29] "OPTIONS /api/estimador HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2019 10:14:29] "POST /api/estimador HTTP/1.1" 200 -
